In [1]:
import logging
from dotenv import load_dotenv

load_dotenv()

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("user",
         """자기소개서 문항: {question}
         위 문항에 대해 답변을 작성해 주세요.
         """),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [4]:
from typing import Sequence, Tuple, List
import math
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langgraph.graph import START, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    index: int
    messages: Annotated[Sequence[BaseMessage], add_messages]
    question: str
    max_length: int
    lower_bound: int
    additional_instruction: str
    answer_length: int
    prev_missing: int
    prev_excess: int
    resume_file_path: str
    resume: str
    JD_file_path: str
    JD: str
    company_name: str
    company_description: str
    job_title: str
    company_news: List[str] # 회사소식은 실시간 뉴스 검색이 아닌, 직접 수집해서 데이터베이스에 저장하는 게 좋을 듯 혹은 새로운 채용공고가 나오면 그 때, 뉴스를 수집하는 방식이 좋을 듯. + 사용자가 직접 넣을 수 있도록 하는 방법 있음.

workflow = StateGraph(state_schema=State)

In [ ]:
def init_state(state: State):
    lower_bound = math.ceil(state["max_length"] * 0.995)

    return {"index": 0,"additional_instruction": "", "answer_length": 0,"lower_bound": lower_bound, "prev_missing": None, "prev_excess": None}